# W1M3 - ETL 프로세스 구현하기

--- 

#### 학습 목표
웹사이트에서 데이터를 가져와서 요구사항에 맞게 가공하는 ETL 파이프라인을 만듭니다.
- Web Scraping에 대한 이해
- Pandas DataFrame에 대한 이해
- ETL Process에 대한 이해
- Database & SQL 기초



#### 사전지식
##### 시나리오
- 당신은 해외로 사업을 확장하고자 하는 기업에서 Data Engineer로 일하고 있습니다. 경영진에서 GDP가 높은 국가들을 대상으로 사업성을 평가하려고 합니다.
- 이 자료는 앞으로 경영진에서 지속적으로 요구할 것으로 생각되기 때문에 자동화된 스크립트를 만들어야 합니다.

##### 기능요구사항
- IMF에서 제공하는 국가별 GDP를 구하세요. (https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29)
- 국가별 GDP를 확인할 수 있는 테이블을 만드세요.
- 해당 테이블에는 GDP가 높은 국가들이 먼저 나와야 합니다.
- GDP의 단위는 1B USD이어야 하고 소수점 2자리까지만 표시해 주세요.
- IMF에서 매년 2회 이 자료를 제공하기 때문에 정보가 갱신되더라도 해당 코드를 재사용해서 정보를 얻을 수 있어야 합니다.



#### 화면 출력
- GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.
- 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다.



#### 프로그래밍 요구사항
##### 라이브러리 사용
- web scaping은 BeautifulSoup4 라이브러리를 사용하세요.
- 데이터 처리를 위해서 pandas 라이브러리를 사용하세요.
- 로그 기록 시에 datetime 라이브러리를 사용하세요.

##### 코드 가독성
- 코드 가독성을 높이기 위해 1) 주석을 사용해서 설명을 추가하고 2) 함수를 만들어서 가독성과 재사용성을 높이세요.

##### 화일 포맷, 데이터베이스 이름 등
- 추출 (Extract)한 정보는 'Countries_by_GDP.json'라는 이름의 JSON 화일 포맷으로 저장해야 합니다.
- 필요한 모든 작업을 수행하는 'etl_project_gdp.py' 코드를 작성하세요.

##### ETL 프로세스
- ETL 프로세스에 따라 코드를 작성하고 각 단계의 시작과 끝을 로그에 기록하세요.
- 이 모든 처리 과정은 'etl_project_log.txt'라는 로그 화일에 기록되어야 합니다. (로그 화일은 매번 다시 생성하는 것이 아니라 기존 화일에 append되어야 합니다.)
- log는 "time, log" 형식으로 기록하세요. 시간은 'Year-Monthname-Day-Hour-Minute-Second' 포맷에 맞게 표시하세요.



#### 팀 활동 요구사항
- wikipeida 페이지가 아닌, IMF 홈페이지에서 직접 데이터를 가져오는 방법은 없을까요? 어떻게 하면 될까요?
- 만약 데이터가 갱신되면 과거의 데이터는 어떻게 되어야 할까요? 과거의 데이터를 조회하는 게 필요하다면 ETL 프로세스를 어떻게 변경해야 할까요?



#### 추가 요구 사항
##### 코드를 수정해서 아래 요구사항을 구현하세요.
- 추출한 데이터를 데이터베이스에 저장하세요. 'Countries_by_GDP'라는 테이블명으로 'World_Economies.db'라는 데이터 베이스에 저장되어야 합니다. 해당 테이블은 'Country', 'GDP_USD_billion'라는 어트리뷰트를 반드시 가져야 합니다.
    - 데이터베이스는 sqlite3 라이브러리를 사용해서 만드세요.
- 필요한 모든 작업을 수행하는 'etl_project_gdp_with_sql.py' 코드를 작성하세요.

##### 화면 출력
- SQL Query를 사용해야 합니다.
- GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.
- 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다.

---

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

# 로그 기록 함수
def log_message(message):
    timestamp = datetime.datetime.now().strftime('%Y-%b-%d-%H-%M-%S')
    with open('etl_project_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp}, {message}\n")

# Extract: 데이터 크롤링
def extract_gdp_data(url):
    log_message("Starting data extraction")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    rows = table.find_all('tr')
    
    data = []
    # 첫 번째 행 (헤더) 제거 후 데이터 추출
    # ('Country/Territory', 'IMF[1][13]', 'World Bank[14]', 'United Nations[15]'])
    for row in rows[1:]:
        cols = row.find_all(['td', 'th'])
        
        # 'sup' 태그 제거 후 텍스트 추출
        cleaned_cols = []
        for col in cols:
            for sup in col.find_all('sup'):
                sup.decompose()  # 'sup' 태그 제거
            cleaned_cols.append(col.text.strip())
        
        data.append(cleaned_cols)
    return data

# Transform: 데이터 변환
def transform_gdp_data(data):
    log_message("Starting data transformation")
    # 데이터프레임에 필요한 열만 포함하도록 동적 처리
    df = pd.DataFrame(data)
    if len(df.columns) == 7:  # 열 수 확인 후 조건 처리
        df.columns = ['Rank', 'Country', 'GDP (Millions of USD)', 'Year', 'Region', 'Extra1', 'Extra2']
        df = df[['Rank', 'Country', 'GDP (Millions of USD)', 'Region']]  # 필요한 열만 선택
    elif len(df.columns) == 5:  # 기존 열 구조
        df.columns = ['Rank', 'Country', 'GDP (Millions of USD)', 'Year', 'Region']
    else:
        raise ValueError(f"Unexpected number of columns: {len(df.columns)}")

    # 변환 처리
    df['GDP (Millions of USD)'] = pd.to_numeric(df['GDP (Millions of USD)'].str.replace(',', ''), errors='coerce')
    df['GDP (B USD)'] = (df['GDP (Millions of USD)'] / 1000).round(2)
    df = df.drop(columns=['GDP (Millions of USD)'])
    df = df.sort_values(by='GDP (B USD)', ascending=False)
    df_filtered = df[df['GDP (B USD)'] >= 100]
    log_message("Data transformation completed")
    return df, df_filtered


# Load: 데이터 저장 및 출력
def load_gdp_data(df, df_filtered):
    log_message("Starting data loading")
    df.to_json('Countries_by_GDP.json', orient='records', lines=True)
    print("GDP 100B USD 이상인 국가:")
    print(df_filtered[['Country', 'GDP (B USD)']])
    
    regions = df_filtered.groupby('Region')['GDP (B USD)'].apply(lambda x: x.nlargest(5).mean())
    print("\nRegion별 Top 5 국가의 GDP 평균:")
    print(regions)
    log_message("Data loading completed")

# 메인 ETL 함수
def etl_process():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
    try:
        log_message("ETL process started")
        data = extract_gdp_data(url)
        df, df_filtered = transform_gdp_data(data)
        load_gdp_data(df, df_filtered)
        log_message("ETL process completed successfully")
    except Exception as e:
        log_message(f"ETL process failed: {str(e)}")

if __name__ == "__main__":
    etl_process()

GDP 100B USD 이상인 국가:
Empty DataFrame
Columns: [Country, GDP (B USD)]
Index: []

Region별 Top 5 국가의 GDP 평균:
Series([], Name: GDP (B USD), dtype: float64)
